In [2]:
import cv2
import mediapipe as mp
import numpy as np
import math

mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose

def calculate_angle (a,b,c):
    a = np.array(a) # First
    b = np.array(b) # Mid
    c = np.array(c) # End

    rad = np.abs(math.atan2(c[1]-b[1], c[0]-b[0]) - math.atan2(a[1]-b[1],a[0]-b[0]))
    angle=(np.abs(rad* 180.0/np.pi))
    #return angle
    return angle

In [3]:
# set_Matrix
#set_D
D=np.zeros((4,4))
D[0][2]=1
D[1][3]=1
#D[4][6]=1
#D[5][7]=1
#print("D = ")
#print(D)

# D ^n:
def matrixPow(Matrix,n):
    if(type(Matrix)==list):
        Matrix=np.array(Matrix)
    if(n==1):
        return Matrix
    else:
        return np.matmul(Matrix,matrixPow(Matrix,n-1))
#F
def get_F(D,n,t):
    F=0
    for i in range(1, n+1):
        F=F+(matrixPow((D*t),i))/math.factorial(i)
    F=F+np.eye(D.shape[0])
    print("F = ")
    print(F)
    return F

# H 
def get_H(num_measurement,num_state):
    H= np.zeros((num_measurement,num_state))
    print(H)
    for i in range(num_state):
        for j in range(num_measurement):
            if(i==j):
                H[i][j]=1
                break
    return H

# Q :
def get_Q(D,F):
    r=np.zeros((F.shape[1],1))
   # print(F[[F.shape[1]-4],[F.shape[1]-1]])
    #print(F)
   # print("r.shape",r.shape)
    copyline=D.shape[0]
   # print("copyline:",copyline)
    i=0
    j=0
    
    while(i<r.shape[0]):
        if j < copyline:
         #   print("i",i,"j",j)
       #     print(F[[F.shape[1]-4+j],[F.shape[1]-1]])
            r[[i],[0]]=F[[F.shape[1]-4+j],[F.shape[1]-1]]
            j=j+1
            i=i+1
        else:
            j=0

  #      np.array(r)[0][i]=np.array(F)[i][F.shape[1]-1]
    print("r = ")
    print(r)
 #   print("r_T = ")
#    print(r.T)
    Q = r*r.T
#    print("Q =")
#    print (Q)
    
    return Q



In [4]:
# 合并

# 33*4=132
# Z= np.zeros((132,132))
def new_D(D,n): #n number of point
 #   print("D:")
 #   print(D)
    Z= np.zeros((D.shape[0]*n,D.shape[0]*n))
#    print("new D is ",D.shape[0]*n,"*",D.shape[0]*n,"Matrix")
    First_one=0
    Second_one=0
    i = 0
    while(i < Z.shape[0]):  
        
        for j in range(Z.shape[1]):
            if i==j:
                if First_one==0:
                    b=j+2
                    Z[i][b]=1
                    First_one=1
                    
                ##    print("i=",i)
                  #  print("j=",b)
                   # print("第一个 1：")
                    #print(Z)
                    Second_one==0
                    i=i+1
                    break
    
            if First_one==1:
                b=b+1
                Z[i][b]=1
                i=i+3
                First_one=0
                
               ## print("i=",i)
               # print("j=",b)
                #print("第二个 1：")
                #print(Z)
                
                if i>=Z.shape[0]-1 & b>=Z.shape[1]-1:
                    print("new D: ")
                    print(Z)
                    return Z
                b=0
                break
#newD=new_D(D,2)
#F=get_F(newD,1,1)# (D,n,t)
#Q=get_Q(D,F)

In [5]:
## Kalman
def set_kalman():
    kalman = cv2.KalmanFilter(4,2) #（x，y，dx，dy) # x1,y1,dx1,dy1,x2,y3,dx2,dy2
# measurementMatrix H 
    #kalman.measurementMatrix = np.array([[1,0,0,0],[0,1,0,0]],np.float32)
    #get_H(num_measurement,num_state):
    kalman.measurementMatrix = np.array(get_H(2,4),np.float32)
# transitionMatrix F 
    #kalman.transitionMatrix = np.array([[1,0,1,0],[0,1,0,1],[0,0,1,0],[0,0,0,1]],np.float32)
    
    #get_F new_D(D,n): #n number of point
    F=get_F(new_D(D,1),1,1) # (D,n,t)

    kalman.transitionMatrix = np.array(F,np.float32)
# processNoiseCov Q
    kalman.processNoiseCov = np.array([[1,0,0,0],[0,1,0,0],[0,0,1,0],[0,0,0,1]],np.float32) * 0.03
   # Q=get_Q(D,F)
    #kalman.processNoiseCov = np.array(Q,np.float32) * 0.03
    
    print("kalman.measurementMatrix:",kalman.measurementMatrix)
    print("kalman.transitionMatrix: ",kalman.transitionMatrix)
    print("kalman.processNoiseCov:",kalman.processNoiseCov)
    return kalman

In [6]:
def video():
    cap =cv2.VideoCapture(0)
    kalman = set_kalman()
# Curl counter variables
    counter=0
    stage = None
    last_measurement = current_measurement = np.array((2,1),np.float32)
    last_predicition = current_prediction = np.zeros((2,1),np.float32)
    
    # Setup mediapipe instance
    with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
        while cap.isOpened():
            #global current_measurement,measurements,last_measurement,current_prediction,last_prediction
            
            ret,frame = cap.read()
        
        # Recolor image to RGB
            image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            image.flags.writeable = False
        
        #Make detection
            results = pose.process(image)
        
        # Recolor image to BGR
            image.flags.writeable =True
            image= cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        
        # print (results)
        # cv2.imshow("Mediapipe Feed", frame)
        
        # Extrack landmarks
            
        
            if results.pose_landmarks==None:
                continue
                
            landmarks = results.pose_landmarks.landmark

            shoulder = [landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x,landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y]
            elbow = [landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].x,landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].y]
            wrist = [landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].x,landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].y]
                
            last_measurement = current_measurement
            last_prediction = current_prediction
            current_x = landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x
            
            current_y = landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y
            print("current_x:",current_x,"current_y:",current_y)
            
    
    #       current_measurement = np.array([[current_x],[current_y]])
            current_measurement = np.array([[current_x],[current_y]],np.float32)
            print("current_measurement",current_measurement)
            kalman.correct(current_measurement)
            print("new_current_measurement",current_measurement)
    # 
            current_prediction = kalman.predict()
            print("current_prediction",current_prediction)
            
            #new_shoulder = [[float(current_prediction[0]),float(current_prediction[1])]]
            #print('new: ',new_shoulder)
            
            # Calculate angle
            angle=calculate_angle(shoulder,elbow,wrist)
                
                #current_measurement = np.array([angle])
              #  kalman.correct(current_measurement)
              #  current_prediction = kalman.predict()
                
            # Curl counter logic
            if angle > 160:
                stage ="down"
            if angle < 45 and stage == 'down':
                stage ="up"
                counter +=1
               # print (counter)
            
     # Visualize angle
            
            #position=tuple(np.multiply(elbow, [640,480]).astype(int))
            position=(100, 100)
            font =cv2.FONT_HERSHEY_SIMPLEX
            cv2.putText(image, str(counter),position,font,2,(255,255,255),2,cv2.LINE_AA)
                                      
            #print (landmarks)
            
        
        # Render detections
            mp_drawing.draw_landmarks (image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                                  )
            cv2.imshow("Mediapipe Feed", image)

            if cv2.waitKey(10) & 0xFF == ord ('q'):
                break
        
    cap.release()
    cv2.destroyAllWindows() 
    cv2.waitKey(1)                       
             
            

In [7]:
video()

[[0. 0. 0. 0.]
 [0. 0. 0. 0.]]
new D: 
[[0. 0. 1. 0.]
 [0. 0. 0. 1.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]]
F = 
[[1. 0. 1. 0.]
 [0. 1. 0. 1.]
 [0. 0. 1. 0.]
 [0. 0. 0. 1.]]
kalman.measurementMatrix: [[1. 0. 0. 0.]
 [0. 1. 0. 0.]]
kalman.transitionMatrix:  [[1. 0. 1. 0.]
 [0. 1. 0. 1.]
 [0. 0. 1. 0.]
 [0. 0. 0. 1.]]
kalman.processNoiseCov: [[0.03 0.   0.   0.  ]
 [0.   0.03 0.   0.  ]
 [0.   0.   0.03 0.  ]
 [0.   0.   0.   0.03]]


INFO: Created TensorFlow Lite XNNPACK delegate for CPU.


current_x: 0.960871696472168 current_y: 0.714238166809082
current_measurement [[0.9608717 ]
 [0.71423817]]
new_current_measurement [[0.9608717 ]
 [0.71423817]]
current_prediction [[0.]
 [0.]
 [0.]
 [0.]]
current_x: 0.9608769416809082 current_y: 0.721835732460022
current_measurement [[0.96087694]
 [0.72183573]]
new_current_measurement [[0.96087694]
 [0.72183573]]
current_prediction [[0.02798671]
 [0.02102434]
 [0.        ]
 [0.        ]]
current_x: 0.9610999822616577 current_y: 0.7277494668960571
current_measurement [[0.9611    ]
 [0.72774947]]
new_current_measurement [[0.9611    ]
 [0.72774947]]
current_prediction [[0.13004857]
 [0.09832437]
 [0.02570262]
 [0.01946675]]
current_x: 0.9620477557182312 current_y: 0.7288515567779541
current_measurement [[0.96204776]
 [0.72885156]]
new_current_measurement [[0.96204776]
 [0.72885156]]
current_prediction [[0.3680199 ]
 [0.27865812]
 [0.0845478 ]
 [0.06406233]]
current_x: 0.9620901346206665 current_y: 0.7286595106124878
current_measurement [[0

current_x: 1.003831148147583 current_y: 0.6940428614616394
current_measurement [[1.0038311 ]
 [0.69404286]]
new_current_measurement [[1.0038311 ]
 [0.69404286]]
current_prediction [[ 1.0126249 ]
 [ 0.681798  ]
 [ 0.00355391]
 [-0.00370787]]
current_x: 1.000508427619934 current_y: 0.6997705698013306
current_measurement [[1.0005084 ]
 [0.69977057]]
new_current_measurement [[1.0005084 ]
 [0.69977057]]
current_prediction [[ 1.0090578 ]
 [ 0.68865275]
 [ 0.00201243]
 [-0.00142136]]
current_x: 1.0009992122650146 current_y: 0.7098166346549988
current_measurement [[1.0009992 ]
 [0.70981663]]
new_current_measurement [[1.0009992 ]
 [0.70981663]]
current_prediction [[1.0063341e+00]
 [6.9966954e-01]
 [9.8720496e-04]
 [1.2711539e-03]]
current_x: 1.0006816387176514 current_y: 0.7099071741104126
current_measurement [[1.0006816]
 [0.7099072]]
new_current_measurement [[1.0006816]
 [0.7099072]]
current_prediction [[1.0039994e+00]
 [7.0695740e-01]
 [2.6809121e-04]
 [2.5736073e-03]]
current_x: 0.991701006

current_x: 0.988983154296875 current_y: 0.7081848382949829
current_measurement [[0.98898315]
 [0.70818484]]
new_current_measurement [[0.98898315]
 [0.70818484]]
current_prediction [[ 9.9180216e-01]
 [ 6.9859719e-01]
 [ 2.5706396e-03]
 [-4.5994495e-04]]
current_x: 0.9886225461959839 current_y: 0.7058481574058533
current_measurement [[0.98862255]
 [0.70584816]]
new_current_measurement [[0.98862255]
 [0.70584816]]
current_prediction [[9.9250412e-01]
 [7.0239866e-01]
 [2.1661229e-03]
 [4.6253804e-04]]
current_x: 0.9782654643058777 current_y: 0.6975290179252625
current_measurement [[0.97826546]
 [0.697529  ]]
new_current_measurement [[0.97826546]
 [0.697529  ]]
current_prediction [[ 9.8630208e-01]
 [ 6.9999927e-01]
 [ 3.5465116e-04]
 [-1.5698781e-04]]
current_x: 0.9696511030197144 current_y: 0.6862937808036804
current_measurement [[0.9696511]
 [0.6862938]]
new_current_measurement [[0.9696511]
 [0.6862938]]
current_prediction [[ 0.97687083]
 [ 0.6917875 ]
 [-0.00176372]
 [-0.00190063]]
curre

current_x: 0.9603613615036011 current_y: 0.7173587083816528
current_measurement [[0.96036136]
 [0.7173587 ]]
new_current_measurement [[0.96036136]
 [0.7173587 ]]
current_prediction [[ 9.5995319e-01]
 [ 7.2035623e-01]
 [-2.3863242e-05]
 [ 2.0870038e-03]]
current_x: 0.959742546081543 current_y: 0.7173317670822144
current_measurement [[0.95974255]
 [0.71733177]]
new_current_measurement [[0.95974255]
 [0.71733177]]
current_prediction [[ 9.5980555e-01]
 [ 7.2066575e-01]
 [-5.0661663e-05]
 [ 1.7022258e-03]]
current_x: 0.9592229127883911 current_y: 0.7173172235488892
current_measurement [[0.9592229]
 [0.7173172]]
new_current_measurement [[0.9592229]
 [0.7173172]]
current_prediction [[ 9.5941246e-01]
 [ 7.2039998e-01]
 [-1.2478577e-04]
 [ 1.2762188e-03]]
current_x: 0.9589813351631165 current_y: 0.7172915935516357
current_measurement [[0.95898134]
 [0.7172916 ]]
new_current_measurement [[0.95898134]
 [0.7172916 ]]
current_prediction [[ 9.5903426e-01]
 [ 7.1984941e-01]
 [-1.7963382e-04]
 [ 8.807

In [3]:
np.array((2,1),np.float32)

array([2., 1.], dtype=float32)